In [1]:
import numpy as np
import networkx as nx
import functools as ft
import math
import random
from datasets import load_dataset

/home/p4ssenger/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
MUTAG = load_dataset("graphs-datasets/MUTAG")

In [3]:
graphs = []

for i in range(150):
    graph = MUTAG["train"][i]
    egdes = zip(graph["edge_index"][0], graph["edge_index"][1])
    G = nx.Graph()
    G.add_edges_from(egdes)
    graphs.append((G, graph["y"][0]))

In [4]:
DIMENSIONS = 10000


def generateRandomVector(d):
    return np.random.randint(2, size=d)


def similarity(A, B):
    if A.size != B.size:
        raise Exception("A and B have different dimensions.")

    count = 0
    for i in range(0, (A.size - 1)):
        if A[i] == B[i]:
            count += 1

    return count / A.size


def mult(A, B):
    return np.bitwise_xor(A, B)


def applyThreshold(V, n=2):
    if n % 2 == 1:
        threshold = lambda x: 1 if x > n / 2 else 0
    else:
        threshold = (
            lambda x: 1
            if x > n / 2
            else (1 if x == n / 2 and bool(random.getrandbits(1)) else 0)
        )

    vectorized_threshold = np.vectorize(threshold)
    return vectorized_threshold(V)


def arithmeticSumVectors(vectors):
    sum = ft.reduce(lambda x, y: x + y, vectors)

    return applyThreshold(sum, len(vectors))


def comparison(A, B):
    dividend = sum(A * B)
    divisor = math.sqrt(sum(A**2) * sum(B**2))

    return dividend / divisor


class ItemMemory:
    def __init__(self, vectors=[]):
        self.vectors = vectors

    def addVector(self, label, V):
        self.vectors.append((label, V))

    def count(self):
        return len(self.vectors)

    def cleanup(self, V):
        return max(self.vectors, key=lambda x: comparison(V, x[1]))

In [5]:
def encodeGraph(graph):
    pr = nx.pagerank(graph[0])

    for value in pr.values():
        key = round(value, 5)
        vertices[str(key)] = generateRandomVector(DIMENSIONS)

    edgesHVs = []

    for edge in graph[0].edges:
        v1 = vertices[str(round(pr[edge[0]], 5))]
        v2 = vertices[str(round(pr[edge[1]], 5))]
        E = mult(v1, v2)
        edgesHVs.append(E)

    return arithmeticSumVectors(edgesHVs)

In [6]:
memory = ItemMemory()
vertices = dict()
labels = [0, 1]

labelsHVs = []
for label in labels:
    graphsHVs = []

    for graph in filter(lambda x: x[1] == label, graphs):
        graphsHVs.append(encodeGraph(graph))

    labelsHVs.append(arithmeticSumVectors(graphsHVs))

memory.addVector("0", labelsHVs[0])
memory.addVector("1", labelsHVs[1])

In [7]:
testGraphs = []

for i in range(30):
    graph = MUTAG["train"][i + 150]
    egdes = zip(graph["edge_index"][0], graph["edge_index"][1])
    G = nx.Graph()
    G.add_edges_from(egdes)
    testGraphs.append((G, graph["y"][0]))

for testGraph in testGraphs:
    print(memory.cleanup(encodeGraph(testGraph))[0] == testGraph[1])

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
